In [3]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time
import sys
sys.path.insert(0, '../')

from datetime import timedelta

In [4]:
from config_paper import key_id, secret_key
from cross_signal import *

In [5]:
#The url at the end is what makes a paper trading bot
base_url='https://paper-api.alpaca.markets'
api = tradeapi.REST(key_id, secret_key,base_url)

In [6]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdate

'2020-04-28'

In [7]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        dataframe['mfi'] = ta.MFI(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'), timeperiod=14)
        dataframe['ultosc'] = ta.ULTOSC(dataframe['high'],dataframe['low'],dataframe['close'])
        
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=20, price='close')
        dataframe['adx'] = tab.ADX(dataframe)

        # required for graphing

        return dataframe

In [8]:
account = api.get_account()
print(account.status)
account.buying_power

ACTIVE


'120000'

In [9]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [10]:
barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
assetsToTrade = [x.upper() for x in symbols]
positionSizing = 0.01

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)
trade=True

In [11]:
trade=True
while trade==True:
        
    for i in range(assetListLen) :
        try:
            symbol = assetsToTrade[i]

            returned_data = api.polygon.historic_agg_v2(symbol, 1, 'minute',_from=startdate, to=startdate).df
            #api.get_barset(symbol,barTimeframe,limit=150).df


            # Calculated trading indicators
            #populate exponential moving averages
            data=populateindicators(returned_data) 


             # Create the buy signal here
            if data['adx'][-1] > 22 and \
            data['mfi'][-1] < 25 and \
            data['ultosc'][-1] < 25 and \
            data['rsi'][-1] < 33 and \
            data['minusdi'][-1] > 25 and \
            data['minusdi'][-1] > data['plusdi'][-1]:

                buyord=True
                assetlist=api.list_positions()
                for asset in assetlist:
                    # if asset is in the list of positions we have already bought, then do not buy again
                    if asset.symbol == symbol:
                        buyord=False
                        print(asset.symbol,asset.qty)
                    
                    
                    
                        #if asset is in list of positions AND is more than 2.5% loss
                        # sell and turn that symbol to null
                        if float(asset.unrealized_intraday_plpc)< -.025 :
                            openPosition = api.get_position(symbol)

                            returned = api.submit_order(symbol,int(openPosition.qty),"sell","market","day")
                            assetsToTrade[i]= symbol+' nulled for below -2.5% pnl'
                            
                #if asset is not already bought, then buy             
                if buyord==True:
                    #cashBalance = account.daytrading_buying_power
                    cashBalance = float(account.buying_power)

                    targetPositionSize = int(cashBalance / (data['close'][-1] / positionSizing))

                    returned = api.submit_order(symbol,targetPositionSize,"buy","market","day") # Market order to open position
                    print('Buy ',returned)

       #Sell conditions 
            elif data['adx'][-1] > 22 and \
                data['rsi'][-1] > 75:
                # Closes position
                openPosition = api.get_position(symbol)

                returned = api.submit_order(symbol,int(openPosition.qty),"sell","market","day") # Market order to fully close position
                print('Sell',returned)
        

        except:
                print(symbol)
    
    
    #get current time and shut off trading
    currenthour=time.localtime().tm_hour 
    if currenthour >= 12: 
        trade=False
        print('Good work, Turning off')
    
    #interval number of seconds to wait
    time.sleep(20)
    
    

AES
BRKB
BFB
KMX
ETFC
MKTK
NI
Buy  Order({   'asset_class': 'us_equity',
    'asset_id': 'c29be0fc-a7a5-463f-b5cc-fe31ede2f86f',
    'canceled_at': None,
    'client_order_id': '51308415-5af8-4666-a3bd-f0ec25609544',
    'created_at': '2020-04-29T06:47:15.163186Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '70025073-2753-45db-8862-f0af41f6b5f6',
    'legs': None,
    'limit_price': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '58',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-04-29T06:47:15.146422Z',
    'symbol': 'JWN',
    'time_in_force': 'day',
    'type': 'market',
    'updated_at': '2020-04-29T06:47:15.163186Z'})
PWR
RTN
UTX
Buy  Order({   'asset_class': 'us_equity',
    'asset_id': '357a8404-f368-40a1-9356-3eb500f4f08a